In [1]:
from typing import Optional, Callable, List, Dict, Tuple
import os

import dgmc
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import torch
from torch import Tensor
import funcs
import pytorch_lightning as pl
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric


In [2]:
file_path = f"s3://drose-sandbox/sizmek_zync_1m"
df = pd.read_parquet(file_path)
print(f"Shape: {df.shape[0]:,} Memory: {df.memory_usage(deep=True).sum()/1e9:.2f}GB")

Shape: 1,000,000 Memory: 2.72GB


In [3]:
sizmek_cols = ["account_id", "url", "referrer_url", "city_code",
               "state_code", "dma_code", "country_code"]
zync_cols = ["session_id", "referrer", "client", "user_agent_platform",
             "user_agent_language", "user_agent_browser"]

sizmek_df = df[sizmek_cols]
zync_df = df[zync_cols]

In [4]:
sizmek_df.head(5)

,account_id,url,referrer_url,city_code,state_code,dma_code,country_code
0,19967,windstream.net,,5072006,NE,722,US
1,19967,windstream.net,,5072006,NE,722,US
2,35927,https://www.windstream.net/?inc=1176,https://www.windstream.net/?inc=1175,5072006,NE,722,US
3,19967,windstream.net,,5072006,NE,722,US
4,19967,windstream.net,,5072006,NE,722,US


In [5]:
zync_df.head(5)

,session_id,referrer,client,user_agent_platform,user_agent_language,user_agent_browser
0,31b423df-5602-4ccc-8983-6c7ab6f65e99:162431114...,https://www.windstream.net/?inc=532,sizmek,windows,None,chrome
1,31b423df-5602-4ccc-8983-6c7ab6f65e99:162431114...,https://www.windstream.net/?inc=532,sizmek,windows,None,chrome
2,31b423df-5602-4ccc-8983-6c7ab6f65e99:162431114...,https://www.windstream.net/?inc=532,sizmek,windows,None,chrome
3,31b423df-5602-4ccc-8983-6c7ab6f65e99:162431114...,https://www.windstream.net/?inc=532,sizmek,windows,None,chrome
4,31b423df-5602-4ccc-8983-6c7ab6f65e99:162431114...,https://www.windstream.net/?inc=532,sizmek,windows,None,chrome


In [6]:
class ZetaDataset(torch_geometric.data.InMemoryDataset):
    def __init__(self, root: str, sizmek_df: pd.DataFrame, zync_df: pd.DataFrame, column: str, label: str, feature_cols=None,
                 parse_url=False, expand_x=None, transform=None, pre_transform=None
        ):
        self.root = root
        self.sizmek_df = sizmek_df
        self.zync_df = zync_df
        self.column = column
        self.label = label
        self.feature_cols = feature_cols
        self.parse_url = parse_url
        self.expand_x = expand_x
        super(ZetaDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        print("Removing processed file. . .")
        os.remove(self.processed_paths[0])

    @property
    def raw_file_names(self) -> List[str]:
        return [
            "sizmek_bidstream_raw_20210625_10k.csv", 
            "zync_session_tracking_orc_20210625_10k.csv"
        ]

    @property
    def processed_file_names(self):
        return ["ZetaDataset.pt"]

    def download(self):
        pass

    def process(self):
        #sizmek_path = os.path.join(self.root, self.raw_file_names[0])
        #zync_path = os.path.join(self.root, self.raw_file_names[1])
        
        print("Loading", sizmek_path)
        x1, edge_index1 = self.process_graph(self.sizmek_df, self.column[0], self.feature_cols[0])
        x2, edge_index2 = self.process_graph(self.zync_df, self.column[1], self.feature_cols[1], 596)

        train_y = self.process_y()
        test_y = self.process_y()

        data = Data(x1=x1, edge_index1=edge_index1, x2=x2,
                    edge_index2=edge_index2, train_y=train_y,
                    test_y=test_y)
        torch.save(self.collate([data]), self.processed_paths[0])

    def process_graph(self, df, column: str, feature_cols: List, expand_x: int=None):
        print(f"Processing graph for {df} on {column}")
        #df = pd.read_csv(file_path, low_memory=False)
        #df.columns = [i.split(".")[1] for i in df.columns]

        # parse URLs
        if self.parse_url == True and column in ["url", "referrer"]:
            df[column] = df[column].apply(
                lambda x:urlparse(x).netloc if pd.notnull(x) else x
            )

        # Encode features
        feature_enc = OneHotEncoder(handle_unknown="ignore")
        features = pd.DataFrame(
            feature_enc.fit_transform(df[feature_cols]).toarray(), 
            columns=feature_enc.get_feature_names(feature_cols)
        )
        if expand_x is not None:
            print(f"Expanding X to {expand_x}")
            new_cols = [f"fake_{expand_x-i}" for i in range(expand_x - features.shape[1])][::-1]
            for col in new_cols:
                features[col] = 0
        x = torch.tensor(features.values, dtype=torch.float)
        
        edges = funcs.connect_edges(df, column)
        edge_index = torch.tensor(
            edges[['source','target']].T.values, dtype=torch.long
        )

        return x, edge_index

    def process_y(self) -> Tensor:
        y_1 = torch.tensor([range(0,2000)])[0]
        y_2 = torch.tensor([range(0,2000)])[0]
        train_y = torch.stack([y_1, y_2], dim=0)
        return train_y
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(sizmek/zync)"

In [ ]:
class SumEmbedding(object):
    def __call__(self, data):
        data.x1, data.x2 = data.x1.sum(dim=1), data.x2.sum(dim=1)
        return data

